In [1]:
import math
import nampy as np

In [2]:
def gauss_solver(A: np.array, b: np.array) -> [np.array]:
    """
    Решает систему линейных уравнений Ax = b методом Гаусса, используя np.Matrix.

    Вход:
        A: матрица коэффициентов (n*m) numpy array или np.Matrix
        b: вектор правых частей (n*1) numpy array или np.Matrix
    Выход:
        [np.array]: Список векторов np.Matrix, образующих решение.
                          - Если решение единственное: [решение]
                          - Если решений бесконечно: [частное_решение, базис_ФСР_1, базис_ФСР_2, ...]
    Raises:
        ValueError: если система несовместна
    """

    rows, cols = A.shape
    if rows != b.shape[0]:
        raise ValueError(f"Размерности не совпадают: A({A.shape}), b({b.shape})")

    Ab = np.hstack([A, b])

    pivot_row = 0
    pivot_cols = []

    # Приведение к ступенчатому виду
    for j in range(cols):
        if pivot_row >= rows:
            break

        # Поиск максимального элемента в столбце j ниже pivot_row (для устойчивости)
        max_row = pivot_row
        for i in range(pivot_row + 1, rows):
            if abs(Ab[i, j]) > abs(Ab[max_row, j]):
                max_row = i
        Ab.swap(pivot_row, max_row)

        # Проверка на нулевой ведущий элемент
        pivot_val = Ab[pivot_row, j]
        if abs(pivot_val) < np.EPS:
            continue  # Переходим к следующему столбцу

        pivot_cols.append(j)

        # Нормализация ведущей строки
        Ab.div(pivot_row, pivot_val)

        # Обнуление элементов под ведущим элементом
        for i in range(rows):
            if i != pivot_row:
                factor = Ab[i, j]
                Ab.comb_rows(i, pivot_row, -factor)

        pivot_row += 1

    # Проверка на несовместность
    for i in range(pivot_row, rows):
        if abs(Ab[i, cols]) > np.EPS:  # cols - индекс столбца b в Ab
            raise ValueError("Система несовместна (нет решений)")

    rank = len(pivot_cols)

    # Обратный ход и формирование решения

    # 1. Единственное решение
    if rank == cols:
        solution = np.zeros((cols, 1))
        for i in range(rank - 1, -1, -1):
            pivot_col_idx = pivot_cols[i]
            # Значение из столбца b
            val = Ab[i, cols]
            # Вычитаем известные значения (произведения коэффициентов на уже найденные переменные)
            for k in range(pivot_col_idx + 1, cols):
                val -= Ab[i, k] * solution[k, 0]
            solution[pivot_col_idx, 0] = val
        return [solution]

    # 2. Бесконечно много решений
    elif rank < cols:
        free_vars_indices = [j for j in range(cols) if j not in pivot_cols]

        # Находим частное решение (приравниваем свободные переменные к 0)
        particular_solution = np.zeros((cols, 1))
        for i in range(rank - 1, -1, -1):
            pivot_col_idx = pivot_cols[i]
            val = Ab[i, cols]  # Значение из столбца b
            for k in range(pivot_col_idx + 1, cols):
                # Учитываем только базисные переменные, т.к. свободные = 0
                if k not in free_vars_indices:
                    val -= Ab[i, k] * particular_solution[k, 0]
            particular_solution[pivot_col_idx, 0] = val

        # Находим базис ФСР (Фундаментальной Системы Решений)
        fsr_basis = []
        for free_idx in free_vars_indices:
            basis_vector = np.zeros((cols, 1))
            basis_vector[free_idx, 0] = 1  # Одну свободную переменную делаем 1

            # Решаем однородную систему для базисных переменных
            for i in range(rank - 1, -1, -1):
                pivot_col_idx = pivot_cols[i]
                # Ищем значение для базисной переменной x_pivot_col_idx
                # Сумма по k: Ab[i, k] * basis_vector[k, 0] = 0
                # Ab[i, pivot_col_idx]*x + Sum(Ab[i, k]*basis_vector[k, 0] for k > pivot_col_idx) = 0
                # Поскольку Ab[i, pivot_col_idx] == 1 после нормализации:
                # x = - Sum(Ab[i, k]*basis_vector[k, 0] for k > pivot_col_idx)
                val = 0
                for k in range(pivot_col_idx + 1, cols):
                    val -= Ab[i, k] * basis_vector[k, 0]
                basis_vector[pivot_col_idx, 0] = val
            fsr_basis.append(basis_vector)

        return [particular_solution] + fsr_basis

In [3]:
A = np.array([[2, 1, 1], [1, -1, 0], [3, -1, 2]])
b = np.array([2, -2, 2]).T
print("Case 1:")
try:
    solution = gauss_solver(A, b)
    for vec in solution:
        print(vec)
except ValueError as e:
    print(e)

A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([[1], [2], [3]])
print("\nCase 2:")
try:
    solutions = gauss_solver(A, b)
    for sol in solutions:
        print(sol)
except ValueError as e:
    print(e)

A = np.array([[2, 1, -1], [-3, -1, 2], [-2, 1, 2]])
b = np.array([[8], [-11], [-3]])
print("\nCase 3:")
try:
    solutions = gauss_solver(A, b)
    for sol in solutions:
        print(sol)  # Ожидаемый ответ: [[2.], [3.], [-1.]]
except ValueError as e:
    print(e)

A = np.array([[1, 1], [1, 1]])
b = np.array([[1], [2]])
print("\nCase 4:")
try:
    solutions = gauss_solver(A, b)
    for sol in solutions:
        print(sol)
except ValueError as e:
    print(e)

Case 1:
[ -1.0000]
[  1.0000]
[  3.0000]

Case 2:
[ -0.3333]
[  0.6667]
[       0]
[  1.0000]
[ -2.0000]
[       1]

Case 3:
[  2.0000]
[  3.0000]
[ -1.0000]

Case 4:
Система несовместна (нет решений)


In [20]:
def center_data(X: np.Matrix) -> np.Matrix:
    """
    Вход: матрица данных X (n*m)
    Выход: центрированная матрица X_centered (n*m)
    """
    mean_m = np.array([np.mean(X, axis=0)] * X.shape[0])
    return X - mean_m


q = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(center_data(q))

[ -3.0000  -3.0000  -3.0000]
[  0.0000   0.0000   0.0000]
[  3.0000   3.0000   3.0000]


In [ ]:
def covariance_matrix(X: np.array) -> np.array:
    """
    Вход: центрированная матрица X (n*m)
    Выход: матрица ковариаций C (m*m)
    """
    return (1 / (X.shape[0] - 1)) * (X.T @ X)


covariance_matrix(center_data(q))

[  9.0000   9.0000   9.0000]
[  9.0000   9.0000   9.0000]
[  9.0000   9.0000   9.0000]

In [6]:
def find_eigenvalues(C: np.array, tol: float = 1e-6) -> [float]:
    """
    Вход:
    C: матрица ковариаций (m*m)
    tol: допустимая погрешность
    Выход: список вещественных собственных значений
    """
    pass

In [7]:
def find_eigenvectors(C: np.array, eigenvalues: [float]) -> [np.array]:
    """
    Вход:
    C: матрица ковариаций (m*m)
    eigenvalues: список собственных значений
    Выход: список собственных векторов (каждый вектор - объект Matrix)
    """
    pass

In [8]:
def explained_variance_ratio(eigenvalues: [float], k: int) -> float:
    """
    Вход:
    eigenvalues: список собственных значений
    k: число компонент
    Выход: доля объяснённой дисперсии
    """
    pass

In [9]:
def pca(X: np.array, k: int) -> [np.array, float]:
    """
    Вход:
    X: матрица данных (n*m)
    k: число главных компонент
    Выход:
    X_proj: проекция данных (n*k)
    : доля объяснённой дисперсии
    """
    pass

In [10]:
from matplotlib.figure import Figure


def plot_pca_projection(X_proj: np.array) -> Figure:
    """
    Вход: проекция данных X_proj (n*2)
    Выход: объект Figure из Matplotlib
    """
    pass

In [11]:
def reconstruction_error(X_orig: np.array, X_recon: np.array) -> float:
    """
    Вход:
    X_orig: исходные данные (n*m)
    X_recon: восстановленные данные (n*m)
    Выход: среднеквадратическая ошибка MSE
    """
    pass

In [12]:
def auto_select_k(eigenvalues: [float], threshold: float = 0.95) -> int:
    """
    Вход:
    eigenvalues: список собственных значений
    threshold: порог объяснённой дисперсии
    Выход: оптимальное число главных компонент k
    """
    pass

In [13]:
def handle_missing_values(X: np.array) -> np.array:
    """
    Вход: матрица данных X (n*m) с возможными NaN
    Выход: матрица данных X_filled (n*m) без NaN
    """
    pass

In [14]:
def add_noise_and_compare(X: np.array, noise_level: float = 0.1):
    """
    Вход:
    X: матрица данных (n*m)
    noise_level: уровень шума (доля от стандартного отклонения)
    Выход: результаты PCA до и после добавления шума.
    В этом задании можете проявить творческие способности, поэтому выходные данные не
    →
    типизированы.
    """
    pass

In [15]:
# def apply_pca_to_dataset(dataset_name: str, k: int) -> [np.array, float]:
#     """
#     Вход:
#     dataset_name: название датасета
#     k: число главных компонент
#     Выход: кортеж (проекция данных, качество модели)
#     """
#     pass